In [1]:
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from scipy import interpolate
import cv2
from copy import copy, deepcopy

In [2]:
# Befehl muss zweimal ausgeführt werden (komisch)
%matplotlib qt
%matplotlib qt

In [3]:
path_folder = 'C:/Users/User/GridTransformation/' # xml path
path_folder, dirs, files = next(os.walk(path_folder))
number_images = len(files)

print(path_folder)
print(files)
print(number_images)

C:/Users/User/GridTransformation/
['Apfel.jpg', 'grid_transformation.ipynb']
2


In [4]:
with Image.open(path_folder + files[0]) as file:
    img = np.array(file)
    x,y,z = np.shape(img)
# print(x,y,z)   

Image <br>
https://help.geogebra.org/topic/nonlinear-image-transformations <br>
More translation <br>
http://www.pacifict.com/WhatsNew.html <br>

In [5]:
plt.close('all')

In [6]:
plt.figure('original')
plt.imshow(img)

In [7]:
z0 = np.array(range(x)) # Imaginär
z1 = np.array(range(y)) # Real
z_transform = np.array(np.empty((x,y,3)))
z_transform.fill(np.nan) # nan werden später interpoliert
# Dient nur zur Veranschaulichung
# z_zero_plot = np.array([[[0 for k in range(3)] for j in range(y)] for i in range(x)]) 
tmp = [[0 for j in range(y)] for i in range(x)]
a = 50                 # 5, passiert gar nichts, wenn n = 0
n = 15                 # 15
m = 1.09              # Potenz von square
func = 'sin'


# print(z0)
# print(z1)
# print(np.shape(z_transform))
# print(np.shape(tmp))

In [8]:
def grid_transform_apple(i,k,a,n):
    z = z0[i]*1j + z1[k]
    return z + np.exp(-(1/2**n)*(abs(z))**2) * (z - a)

def grid_transform_square(i,k,m):
    z = (z0[i]*1j + z1[k])/255
    return z**z *255

def grid_transform_sin(i,k):
    z = (z0[i]*1j + z1[k])/255
    return 1.2*np.sin(z)*255

In [9]:
for i in range(x):
    for k in range(y):
        if func == 'apple':
            tmp[i][k] = grid_transform_apple(i,k,a,n)
        elif func == 'square':
            tmp[i][k] = grid_transform_square(i,k,m)
        elif func == 'sin':
            tmp[i][k] = grid_transform_sin(i,k)
             
        complex_number = tmp[i][k]
        complex_imag = round(int(complex_number.imag))
        complex_real = round(int(complex_number.real))
        
        if complex_imag >= 0 and complex_imag < x:
            if complex_real >= 0 and complex_real < y:
                z_transform[complex_imag][complex_real] = img[i][k]

isnan(): <br>
https://stackoverflow.com/questions/5124376/convert-nan-value-to-zero <br>

In [10]:
z_transform_mask = deepcopy(z_transform)
z_transform_mask[z_transform_mask>0] = 0
where_are_NaNs = np.isnan(z_transform)
z_transform_mask[where_are_NaNs] = 255
# print(z_transform)
z_transform[where_are_NaNs] = 0

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in greater
  


In [11]:
#plt.figure('z_transform')
#plt.imshow(z_transform/255)
plt.figure('z_transform_mask')
plt.imshow(z_transform_mask/255)

inpaint(): <br>
http://www.bogotobogo.com/python/OpenCV_Python/python_opencv3_Image_reconstruction_Inpainting_Interpolation.php <br>

In [12]:
# cv2.inpaint benötigt arrays vom typ uint8
# print(z_transform_mask.dtype)
# print(z_transform.dtype)

mask = np.uint8(z_transform_mask)
img = np.uint8(z_transform)

# print(mask.dtype)
# print(img.dtype)

dst_TELEA = cv2.inpaint(img, mask[:,:,0], 3, cv2.INPAINT_TELEA)
dst_NS = cv2.inpaint(img, mask[:,:,0], 3, cv2.INPAINT_NS)

In [13]:
# plt.figure('img')
# plt.imshow(mask)

In [14]:
# plt.figure('mask')
# plt.imshow(mask)

In [15]:
# schlechter als dst_NS
# plt.figure('TELEA')
# plt.imshow(dst_TELEA/255)

In [16]:
# Diese Interpolation verwenden
plt.figure('dst_NS')
plt.imshow(dst_NS/255)

In [17]:
np.cos(np.pi/4*z1[k])*50j +z1[k]

(312+50j)

In [18]:
(np.cos(np.pi/4*30))*50j +200

(200-1.3474209693803825e-13j)

In [19]:
np.cos(np.pi/180*30)*50j +200

(200+43.30127018922194j)

In [20]:
np.cos(np.pi/180*31)*50j +200

(200+42.858365035105614j)

In [21]:
a = 226
240*1j + np.cos(np.pi/90*a)*40j + a

(226+238.60402013189994j)